<a href="https://colab.research.google.com/github/lifewinning/MLTSA22_final/blob/main/MLSTA_Final_Project_Preprocessing_with_Timestamps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Initialize EarthEngine, import geemap



In [1]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=x5IHMOoFOtZi8-BYMdRTajXoqLxukF8THkZEgq8OLvw&tc=eIXISSu7qO39FzY7xj-hrX7JtIxDBWFghHRAZUlc2kg&cc=dvDA2CYSR8Dx82iWsIhUKglXdfcmcDBIhCGh5av_Bhw

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWghMf7VApJr3F6VTASZlYFFVfBYv2SYfKZ8pdfZPyZwv5QPm04nTEs

Successfully saved authorization token.


In [2]:
pip install geemap

In [3]:
import geemap
Map = geemap.Map()

In [7]:
import json
from urllib.request import urlopen

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
cd /content/drive/MyDrive/MLTSA22

/content/drive/MyDrive/MLTSA22


# Set date params and load clipping features

In [5]:
# establish time series window
start = ee.Date('2018-01-01')
finish = ee.Date('2021-12-31')

In [8]:
# load geojson with bounding boxes
url = urlopen('https://raw.githubusercontent.com/lifewinning/MLTSA22_final/main/added_ports.geojson')
geojson = json.load(url)
for site, index in enumerate(range(0,4)):
  print(index, geojson['features'][site]['properties']['site'])


0 oakland
1 tacoma
2 shanghai
3 bayonne


# Some generalizing functions

In [9]:
def cliptoaoi(image):
  return image.clip(aoi)

In [10]:
# generate a range of days between start and end of time series
def getdates(collection):
  first = collection.first().date()
  print(first.format('yyy-MM-dd').getInfo())
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1), 6).map(getDays)

In [96]:
def day_mosaics(date, newlist):
  date = ee.Date(date)
  newlist = ee.List(newlist)
  
  filtered = collection_clipped.filterDate(date, date.advance(1,'day'));

  date_formatted = date.format('yyyy-MM-dd_H:m:s', 'UTC')

  image_m = ee.Image(filtered.mosaic()).set({'ID': date_formatted}).set({'date': date}).clip(aoi)

  # Add the mosaic to a list only if the collection has images
  return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image_m), newlist))

In [ ]:
def check_gaps(list):
  collection_list

In [ ]:
def get_thumbnails(coords, ):
  aoi = ee.Geometry.Polygon(coords)
  # query EE
  collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
  collection_list = collection_clipped.toList(collection.size())

  # mosaic images from the same day as needed
  # newcol = ee.ImageCollection(ee.List(getdates(collection).iterate(day_mosaics, ee.List([]))))
  # newcol_list = newcol.toList(newcol.size())
  ids = collection_clipped.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('Y-MM-dd_H:m:s', 'US/Eastern')).getInfo()



# Bayonne

In [79]:
coords = geojson['features'][3]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection.size())


# mosaic images from the same day as needed
# newcol = ee.ImageCollection(ee.List(getdates(collection).iterate(day_mosaics, ee.List([]))))
# newcol_list = newcol.toList(newcol.size())
ids = collection_clipped.aggregate_array('system:time_start').map(lambda time_start: ee.Date(time_start).format('Y-MM-dd_H:m:s', 'US/Eastern')).getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [81]:
image = collection_clipped.first()
out_img = "tests/bayonne.png"
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [82]:
out_dir = 'timestampedit/bayonne'
geemap.get_image_collection_thumbnails(collection_clipped, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 120



## Tacoma

In [88]:
coords = geojson['features'][1]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

In [66]:
# generate a range of days between start and end of time series
def getdates(coll):
  first = coll.first().date()
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1), 6).map(getDays)

In [93]:
# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()
# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [94]:
print(ids)

['2018-01-07_6:12:38', '2018-01-19_6:12:38', '2018-04-01_7:12:38', '2018-04-13_7:12:38', '2018-04-25_7:12:38', '2018-05-07_7:12:38', '2018-05-19_7:12:38', '2018-05-31_7:12:38', '2018-06-12_7:12:38', '2018-06-24_7:12:38', '2018-07-06_7:12:38', '2018-07-18_7:12:38', '2018-07-30_7:12:38', '2018-08-11_7:12:38', '2018-08-23_7:12:38', '2018-09-04_7:12:38', '2018-09-16_7:12:38', '2018-09-28_7:12:38', '2018-10-10_7:12:38', '2018-10-22_7:12:38', '2018-11-03_7:12:38', '2018-11-15_6:12:38', '2018-11-27_6:12:38', '2018-12-09_6:12:38', '2018-12-21_6:12:38', '2019-01-02_6:12:38', '2019-01-14_6:12:38', '2019-01-26_6:12:38', '2019-02-07_6:12:38', '2019-02-19_6:12:38', '2019-03-03_6:12:38', '2019-03-15_7:12:38', '2019-03-27_7:12:38', '2019-04-08_7:12:38', '2019-04-20_7:12:38', '2019-05-02_7:12:38', '2019-05-14_7:12:38', '2019-05-26_7:12:38', '2019-06-07_7:12:38', '2019-06-19_7:12:38', '2019-07-01_7:12:38', '2019-07-13_7:12:38', '2019-07-25_7:12:38', '2019-08-18_7:12:38', '2019-08-30_7:12:38', '2019-09-

In [59]:
# take a quick look at the first image just to see
out_img = "tests/tacoma.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [95]:
out_dir = 'timestampedit/tacoma'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 114



# Rotterdam

In [ ]:
# generate a range of days between start and end of time series
def getdates(coll):
  diff = finish.difference(start, 'day')
  def getDays(day):
    return start.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1),6).map(getDays)

In [ ]:
# update to rotterdam
coords = geojson['features'][3]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [ ]:
len(ids), ids

(239,
 ['2018-01-01',
  '2018-01-07',
  '2018-01-13',
  '2018-01-19',
  '2018-01-25',
  '2018-01-31',
  '2018-02-06',
  '2018-02-12',
  '2018-02-18',
  '2018-02-24',
  '2018-03-02',
  '2018-03-08',
  '2018-03-14',
  '2018-03-20',
  '2018-03-26',
  '2018-04-01',
  '2018-04-07',
  '2018-04-13',
  '2018-04-19',
  '2018-04-25',
  '2018-05-01',
  '2018-05-07',
  '2018-05-13',
  '2018-05-19',
  '2018-05-25',
  '2018-05-31',
  '2018-06-06',
  '2018-06-12',
  '2018-06-18',
  '2018-06-24',
  '2018-06-30',
  '2018-07-06',
  '2018-07-12',
  '2018-07-18',
  '2018-07-24',
  '2018-07-30',
  '2018-08-05',
  '2018-08-11',
  '2018-08-17',
  '2018-08-23',
  '2018-09-04',
  '2018-09-10',
  '2018-09-16',
  '2018-09-22',
  '2018-09-28',
  '2018-10-04',
  '2018-10-10',
  '2018-10-16',
  '2018-10-22',
  '2018-10-28',
  '2018-11-03',
  '2018-11-09',
  '2018-11-15',
  '2018-11-21',
  '2018-11-27',
  '2018-12-03',
  '2018-12-09',
  '2018-12-15',
  '2018-12-21',
  '2018-12-27',
  '2019-01-02',
  '2019-01-08',
  

In [ ]:
# take a quick look at the first image just to see
out_img = "tests/rotterdam-first.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
geemap.show_image(out_img)

Output()

In [ ]:
out_dir = 'finalimages/rotterdam'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=500, format="png", names=ids)

Total number of images: 239



# Singapore

In [ ]:
# update to singapore
coords = geojson['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

# query EE
collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate(start, finish).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
collection_clipped = collection.map(cliptoaoi).sort('segmentStartTime')
collection_list = collection_clipped.toList(collection_clipped.size())

In [ ]:
# generate a range of days between start and end of time series
# because of the size of the image and gaps in overlap days (?) Singapore actually has a 12 day lag
def getdates(coll):
  first = ee.Date('2018-01-06')
  diff = finish.difference(first, 'day')
  def getDays(day):
    return first.advance(day,'day')
  
  return ee.List.sequence(0, diff.subtract(1),12).map(getDays)

In [ ]:
# mosaic images from the same day as needed, make list of IDs from mosaic images
newcol = ee.ImageCollection(ee.List(getdates(collection_clipped).iterate(day_mosaics, ee.List([]))))
newcol_list = newcol.toList(newcol.size())
ids = newcol.aggregate_array('ID').getInfo()

# this is for exporting
vis_params = {'min': -20, 'max': 0, 'bands':'VH'}

In [ ]:
# take a quick look at the first image just to see
out_img = "tests/singapore-first.png"
image = newcol.first()
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=800, format='png')
geemap.show_image(out_img)

Output()

In [ ]:
out_dir = 'finalimages/singapore'
geemap.get_image_collection_thumbnails(newcol, out_dir, vis_params, dimensions=800, format="png", names=ids)

Total number of images: 118

